(https://www.kaggle.com/c/ashrae-energy-prediction/)

# ASRAE - Modeling

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly as plt
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from math import sqrt

import glob

import pandas_profiling as ppf
import sweetviz as sv
import missingno as msno

import pprint
import datetime
from math import sqrt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from prophet import Prophet

import xgboost as xgb


import pmdarima as pm
from pmdarima import model_selection
from pmdarima import pipeline
from pmdarima import preprocessing as ppc
from pmdarima import arima
from pmdarima.arima import ndiffs
from pmdarima.arima import ADFTest

print("pmdarima version: %s" % pm.__version__)

#import tools as tl

sns.set(rc={'figure.figsize':(30,20)})

pd.options.display.float_format = '{:.2f}'.format
pd.set_option("precision", 2)

plt.rcParams.update({'font.size': 22})
sns.set(font_scale = 2)

import pprint
pp = pprint.PrettyPrinter(indent=4)

import warnings
warnings.filterwarnings('ignore')

pmdarima version: 1.8.2


In [2]:
print(xgb.__version__)

1.4.2


In [3]:
pd.options.plotting.backend = "matplotlib"

In [4]:
pwd

'G:\\My Drive\\Code\\Springboard_Capstone_Energy'

In [5]:
# Drop Specific columns
def building_meter_drop(df):
    return df.drop(columns=['precip_depth_1_hr','wind_direction','wind_speed','dew_temperature','year_built','floor_count','cloud_coverage'],axis=1)

In [6]:
# Interpolate Missing Data
def building_meter_interp(df):
    df['sea_level_pressure'] = df['sea_level_pressure'].interpolate(method='time')
    df['meter_reading'] = df['meter_reading'].interpolate(method='time')
    df['air_temperature'] = df['air_temperature'].interpolate(method='time')
    return df

In [7]:
# Aggregate remaining Data
def building_meter_agg(df,resamp_str):
    df_samp = df
    agg_dict={'meter_reading':'mean', 
                  'air_temperature':'max',
                  'building_id': 'max',
                  'site_id': 'max',
                  'square_feet': 'max',
                  'day_of_week':'max',
                  'weekend':'max',
                  'month':'max',
                  'season':'max'
                  }

    # Resample Dataframe
    df = df.resample(resamp_str).agg(agg_dict)
    
    return df

## See Statistics written for each file (building-meter) combo

In [8]:
# See Building Statistics for missing data
df_stats = pd.read_csv('.//data_clean//building_reading_stats.csv')
# Keep only data that has at least 50 % of it

In [9]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   file_name                     2380 non-null   object 
 1   building                      2380 non-null   int64  
 2   meter                         2380 non-null   int64  
 3   len_train                     2380 non-null   int64  
 4   len_zero                      2380 non-null   int64  
 5   per_data                      2380 non-null   float64
 6   category                      2380 non-null   object 
 7   corr_meter_to_airtemp         2380 non-null   float64
 8   corr_meter_to_airtemp_wknd    2378 non-null   float64
 9   corr_meter_to_airtemp_wkdy    2380 non-null   float64
 10  corr_meter_to_airtemp_winter  2330 non-null   float64
 11  corr_meter_to_airtemp_spring  2347 non-null   float64
 12  corr_meter_to_airtemp_summer  2344 non-null   float64
 13  cor

In [20]:
# Modeling Choices
# Buildings with 1
df_stats.sort_values(['building','meter'],axis=0,inplace=True)
df_stats.reset_index(drop=True,inplace=True)

In [ ]:
# These represent buildings that are in top, mid and lower in square footage
# choose from these when we model our loop
lrg = [1148]
med = [1063, 1309, 991, 144, 693]
sm = [846, 621, 822, 816, 44]

# lrg = [869, 1148, 375, 365, 269]

bldgno = []
bldgno.extend(lrg)
bldgno.extend(med)
bldgno.extend(sm)

# 0: Electricity 1: chilledwater 2: steam 3: hotwater

In [ ]:
df_stats[(df_stats['building'] == 1148) & df_stats['meter'] == 1]

In [ ]:
# Examine building choices
df_stats[df_stats['building'] == 1148]

### Build list of files to include

# Build 3 Models

In [ ]:
# Data is daily
# # data_sizes = [360] # Start with one year # Train on whole year

output_vis_folder = '.\\visualization\\models\\'

# Control Variables that will run these specific models if True
ARIMA = True
FBProphet = True
xgboost = True

# Show Visualizations
quiet_mode = True

resample = True
backfill = 'mean'
exogen = True

thresh = 50

# Dictionary to hold all the results, will turn into a dataframe
stats_all = {}
stats_list = []

stats = {}

# For each file in the directory
direct = '.\\data_clean\\buildings_by_meter\\'


files = df_stats['file_name']
bl = df_stats['building']

for num,file in enumerate(files):
    print(file)
    
    # Temporary dictionary that represents a row
    stats = {}
    
    # Screen results
    if df_stats.iloc[num]['per_data'] < 80:
        stats['status'] = 'FAIL: Length < 80% data present'
        print('not enough data')
        continue

#     # Skip if building not on our radar
#     if bl[num] not in bldgno:
#         continue
#     else:
#         print('File: ' + file)
    try:
        df = pd.read_pickle(direct + file)
    except FileNotFoundError:
        continue
    
    print('Length on load: ' + str(len(df)))
    stats['Len_load'] = len(df)
    
    if len(df) < 20000:
        stats['status'] = 'FAIL: Length < 20k'
        continue

    # Load key variables in dictionary
    stats['building'] = df.iloc[0]['building_id']
    stats['square_feet'] = df.iloc[0]['square_feet']   
    stats['meter'] = bldg_meter = df.iloc[0]['meter']
    stats['file_name'] = file
 
    # Process Columns
    df = building_meter_drop(df) # Drop poorly correlated columns
    df = building_meter_interp(df) # Time series linear interpolation due to standard time deltas
    df = building_meter_agg(df,'1D') # aggregate remaining columns
    
    print('Length after agg: ' + str(len(df)))
    stats['Len_agg-clean'] = len(df)
    
    # Find out how many lingering NA columns are left
    b4_NA = len(df)
    df.dropna(how='any',inplace=True)
    tot = b4_NA - len(df)
    print('Dropped N/A cols: ' + str(tot))
    
    #exit if we drop nan rows
    if tot > 100:
        stats['status'] = 'FAIL: Drop over 100 Rows'
        continue
    
    print('Length after blank drop: ' + str(len(df)))
    stats['Len_blank'] = len(df)
    
    # Keep track of Models
    model_ctr = 1 # Initialize Model Counter
    model_prefixes = []

    # Split the whole set
    df_train = df.loc['2016-01-01':'2016-09-30']
    df_test = df.loc['2016-10-01':'2016-12-31']
    
    # skip building if we dont have enough data
#     try:
#         if (len(df_train) / len(df_test)) < 2.5:
#             print('Not enough training data')
#         continue
#     except ZeroDivisionError as ze:
#         print('DF empty')
#         continue
      
    X_train = df_train.drop(labels=['meter_reading'],axis=1)
    X_train.fillna(method='ffill')

    X_test = df_test.drop(labels=['meter_reading'],axis=1)
    X_test.fillna(method='ffill')

    y_train = df_train['meter_reading']
    y_train.fillna(method='ffill')

    y_test = df_test['meter_reading']
    y_test.fillna(method='ffill')
    
    # Get rid of constants
    X_train.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
    X_test.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
    
    #   ######################################    ARIMA   ##################################################
    if ARIMA:
#                 # Keep track of model prefix column names by building prefix to number iterations
        model_prefix = 'Model_' + str(model_ctr)
        model_prefixes.append(model_prefix)
        
        # Populate DataFrame with statistics
        stats[model_prefix + '_' + 'Type'] = 'ARIMA_Exogen'
        stats[model_prefix + '_' + 'Description'] = 'ARIMA_Exogen, with daily aggregation'
        stats[model_prefix + '_' + 'DateStart_Train'] = str(X_train.iloc[0].name).split(" ")[0]
        stats[model_prefix + '_' + 'DateEnd_Train'] = str(X_train.iloc[-1].name).split(" ")[0]
        stats[model_prefix + '_' + 'DateStart_Test'] = str(X_test.iloc[0].name).split(" ")[0]
        stats[model_prefix + '_' + 'DateEnd_Test'] = str(X_test.iloc[-1].name).split(" ")[0]
        stats[model_prefix + '_' + 'Len_Train'] = len(X_train)
        stats[model_prefix + '_' + 'Len_Test'] = len(X_test)

        print('ARIMA Model......',end=' ')

        # Try building model
        try:
            model = pm.auto_arima(y_train,X=X_train,trace=False,n_fits=10,seasonal=True,m=7,error_action='ignore')
            stats[model_prefix + '_' + 'Status'] = 'PASS'
        except ValueError as ve:
            print('Model will not converge')
            stats[model_prefix + '_' + 'Status'] = 'FAIL'

        preds, conf_int = model.predict(X=X_test,n_periods=y_test.shape[0], return_conf_int=True)

        predict = pd.Series(preds,index=y_test.index)

        # Keep each slice data here
        df_result = pd.concat([predict,y_test],axis=1)
        df_result.columns = ['meter_predict_ARIMA','meter_actual']

        RMSE_error = (mean_squared_error(df_result['meter_predict_ARIMA'],df_result['meter_actual'],squared=False))

        stats[model_prefix + '_' + 'RMSE'] = RMSE_error
        stats[model_prefix + '_' + 'y_test_MAX'] = np.max(df_result['meter_actual'])
        stats[model_prefix + '_' + 'y_test_MIN'] = np.min(df_result['meter_actual'])
        stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(df_result['meter_actual'])
        stats[model_prefix + '_' + 'y_test_STD'] = np.std(df_result['meter_actual'])
        stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(df_result['meter_predict_ARIMA'])
        stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(df_result['meter_predict_ARIMA'])
        stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(df_result['meter_predict_ARIMA'])
        stats[model_prefix + '_' + 'y_pred_STD'] = np.std(df_result['meter_predict_ARIMA'])

        # RMSE relative to the range of the test range
        stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                             ((np.max(df_result['meter_actual'])) \
                                              - (np.min(df_result['meter_actual']))) \
                                              * 100

        # Create Graph
        plt.figure(figsize=(20,10))
        plt.plot(df_result['meter_predict_ARIMA'],label="ARIMA")
        plt.plot(df_result['meter_actual'],label="Actual")
        plt.legend(loc = 'upper right')      
        plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + model_prefix + '_' + 'ARIMA_Exogen')
        plt.xlabel('Date')
        plt.ylabel('Meter Reading')
        
        plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_ARIMA_Exogen.jpg')
        print('RMSE: ' + str(RMSE_error)) 
          
         ######################################    PROPHET   ##################################################
        if FBProphet:           
            print('Prophet......',end=' ')
            model_ctr += 1 # Increment Model Counter
            model_prefix = 'Model_' + str(model_ctr) # Build string for keys
            model_prefixes.append(model_prefix)

            # Need renaming for Prophet - data prep
            df_prophet_train = df_train.reset_index()
            df_prophet_test = df_test.reset_index()
            
            
            # Get rid of constants
            df_prophet_train.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
            df_prophet_test.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)

            # Timestamp must be ds column
            df_prophet_train = df_prophet_train.rename(columns={'timestamp':'ds','meter_reading':'y'})
            df_prophet_test = df_prophet_test.rename(columns={'timestamp':'ds','meter_reading':'y'})
            
            
            # Populate DataFrame with statistics
            stats[model_prefix + '_' + 'Type'] = 'Prophet_Exogen'
            stats[model_prefix + '_' + 'DateStart_Train'] = str(df_prophet_train.iloc[0]['ds']).split(" ")[0]
            stats[model_prefix + '_' + 'DateEnd_Train'] = str(df_prophet_train.iloc[-1]['ds']).split(" ")[0]
            stats[model_prefix + '_' + 'DateStart_Test'] = str(df_prophet_test.iloc[0].name).split(" ")[0]
            stats[model_prefix + '_' + 'DateEnd_Test'] = str(df_prophet_test.iloc[-1].name).split(" ")[0]           
            stats[model_prefix + '_' + 'Len_Train'] = len(df_prophet_train)
            stats[model_prefix + '_' + 'Len_Test'] = len(df_prophet_test)

            # Initialize Prophet Model
            m = Prophet(daily_seasonality=True)
            m.add_regressor('air_temperature')
            m.add_regressor('day_of_week')
            m.add_regressor('weekend')
            m.add_regressor('month')
            m.add_regressor('season')
            m.fit(df_prophet_train)
            predicts = m.predict(df_prophet_test)

            RMSE_error = (mean_squared_error(predicts['yhat'],df_prophet_test['y'],squared=False))

            stats[model_prefix + '_' + 'RMSE'] = RMSE_error
            stats[model_prefix + '_' + 'y_test_MAX'] = np.max(df_prophet_test['y'])
            stats[model_prefix + '_' + 'y_test_MIN'] = np.min(df_prophet_test['y'])
            stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(df_prophet_test['y'])
            
            # RMSE relative to the range of the test range
            stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                                 ((np.max(df_prophet_test['y'])) \
                                                  - (np.min(df_prophet_test['y']))) \
                                                  * 100

            stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(predicts['yhat'])
            stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(predicts['yhat'])
            stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(predicts['yhat'])


            plt.figure(figsize=(20,10))
            plt.plot(predicts['yhat'],label="Prophet")
            plt.plot(df_prophet_test['y'],label="Actual")
            plt.legend(loc = 'upper right')
            
            plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + \
                      model_prefix + '_' + 'Prophet_Exogen')
            plt.xlabel('Date')
            plt.ylabel('Meter Reading')
            
            plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_Prophet_Exogen.jpg')

            print('RMSE: ' + str(RMSE_error))
                    
#             ######################################    XGBOOST   ##################################################
            if xgboost:
                print('XGBoost......',end=' ')
                model_ctr += 1
                model_prefix = 'Model_' + str(model_ctr)
                model_prefixes.append(model_prefix)

                # Populate DataFrame with statistics
                stats[model_prefix + '_' + 'Type'] = 'XGBoost'
                stats[model_prefix + '_' + 'Description'] = 'ARIMA_Exogen, with daily aggregation'
                stats[model_prefix + '_' + 'DateStart_Train'] = str(X_train.iloc[0].name).split(" ")[0]
                stats[model_prefix + '_' + 'DateEnd_Train'] = str(X_train.iloc[-1].name).split(" ")[0]
                stats[model_prefix + '_' + 'DateStart_Test'] = str(X_test.iloc[0].name).split(" ")[0]
                stats[model_prefix + '_' + 'DateEnd_Test'] = str(X_test.iloc[-1].name).split(" ")[0]
                stats[model_prefix + '_' + 'Len_Train'] = len(X_train)
                stats[model_prefix + '_' + 'Len_Test'] = len(X_test)
    
                # Instantiate Model
                gbm_model = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 1000, seed = 123)
                gbm_model.fit(X_train,y_train)
                predict = gbm_model.predict(X_test)
                
                predict = pd.Series(predict,index=y_test.index)
                
                df_result = pd.concat([predict,y_test],axis=1,ignore_index=True)
                df_result.columns = ['meter_predict_XGBoost','meter_actual']
                df_result.sort_index(inplace=True)
                
                RMSE_error = (mean_squared_error(predict,y_test,squared=False))
                
                stats[model_prefix + '_' + 'RMSE'] = RMSE_error
                stats[model_prefix + '_' + 'y_test_MAX'] = np.max(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_test_MIN'] = np.min(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_test_STD'] = np.std(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(df_result['meter_predict_XGBoost'])
                stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(df_result['meter_predict_XGBoost'])
                stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(df_result['meter_predict_XGBoost'])
                stats[model_prefix + '_' + 'y_pred_STD'] = np.std(df_result['meter_predict_XGBoost'])

                # RMSE relative to the range of the test range
                stats[model_prefix + '_' + 'y_pred_RMSE_pct_Range'] = RMSE_error / \
                                                     ((np.max(df_result['meter_actual'])) \
                                                      - (np.min(df_result['meter_actual']))) \
                                                      * 100
                
                plt.figure(figsize=(20,10))
                plt.plot(df_result['meter_predict_XGBoost'],label="XGBoost")
                plt.plot(df_result['meter_actual'],label="Actual")
                plt.legend(loc = 'upper right')
                plt.title('Bldg ' + str(stats['building']) + ' Meter ' + str(stats['meter']) + ' ' + model_prefix + '_' + 'XGBoost')
                plt.xlabel('Date')
                plt.ylabel('Meter Reading')
                plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_XGBoost.jpg')
                print('RMSE: ' + str(RMSE_error))
                
#             df_viz = pd.DataFrame(df_result['meter_predict_XGBoost'],df_result['meter_predict_ARIMA'],df_result['meter_predict_Prophet'],index=df_result.index)
           

#             plt.plot(df_result['meter_predict_ARIMA'],label="ARIMA")
#             sns.lineplot(data=[df_result['meter_predict_XGBoost'],
#                                 df_result['meter_predict_ARIMA']],
#                          x=df_result.index,
#                          y=)
#             plt.plot(df_result['meter_predict_XGBoost'],label="XGBoost")
#             plt.plot(df_result['meter_actual'],label="Actual")
#             plt.legend(loc = 'upper right')
#             plt.savefig(output_vis_folder + str(stats['building']) + '_' + str(stats['meter']) + '_All.jpg')

#             print(df_viz)

        print()
        print()
    
        stats_list.append(stats)
        # Write dictionary and file every 10 buildings to save results
        if stats['building'] % 10 == 0:
            df_results = pd.DataFrame(stats_list)
            df_results = pd.DataFrame(stats_list)
            df_results.to_csv('Model_Results.csv',float_format="{:,.2f}".format)
            
df_results = pd.DataFrame(stats_list)
df_results.to_csv('Model_Results.csv',float_format="{:,.2f}".format)


0_0.pkl
not enough data
1_0.pkl
not enough data
2_0.pkl
not enough data
3_0.pkl
not enough data
4_0.pkl
not enough data
5_0.pkl
not enough data
6_0.pkl
not enough data
7_0.pkl
not enough data
7_1.pkl
Length on load: 24787
Length after agg: 1037
Dropped N/A cols: 1
Length after blank drop: 1036
ARIMA Model...... RMSE: 2146.7328939178137
Prophet...... 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


RMSE: 3609.4369303123417
XGBoost...... RMSE: 2594.5800580425903


8_0.pkl
not enough data
9_0.pkl
not enough data
9_1.pkl
not enough data
10_0.pkl
not enough data
11_0.pkl
not enough data
12_0.pkl
not enough data
13_0.pkl
not enough data
13_1.pkl
Length on load: 24460
Length after agg: 1037
Dropped N/A cols: 14
Length after blank drop: 1023
ARIMA Model...... 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


RMSE: 1187.222470032455
Prophet...... RMSE: 899.0183857744602
XGBoost...... RMSE: 1321.38366343645


14_0.pkl
not enough data
14_1.pkl
Length on load: 24789
Length after agg: 1037
Dropped N/A cols: 1
Length after blank drop: 1036
ARIMA Model...... 

...

RMSE: 55.664758971500895
Prophet...... RMSE: 88.81610815212971
XGBoost...... RMSE: 33.53660687406568


333_0.pkl
Length on load: 26278
Length after agg: 1096
Dropped N/A cols: 1
Length after blank drop: 1095
ARIMA Model...... 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


RMSE: 18.600994106516787
Prophet...... RMSE: 34.73496282683334
XGBoost...... RMSE: 11.272443142657902


334_0.pkl
Length on load: 26278
Length after agg: 1096
Dropped N/A cols: 1
Length after blank drop: 1095
ARIMA Model...... 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


RMSE: 21.668610205537245
Prophet...... RMSE: 42.14303135618948
XGBoost...... RMSE: 7.929283111339076


335_0.pkl
Length on load: 26302
Length after agg: 1096
Dropped N/A cols: 0
Length after blank drop: 1096
ARIMA Model...... 

In [ ]:
df_stats

In [ ]:
df_stats[df_stats['per_data'] < 75]

In [ ]:
df_stats = df_stats.merge(df_results,how='left',on=['file_name'])

In [ ]:
df_stats.to_csv('All Results.csv',float_format="{:,.2f}".format)

In [ ]:
df_results.iloc[0]

In [ ]:
df_results.filter(regex='RMSE')

In [ ]:
df_results.filter(regex='Model_')
df.to_csv('results.csv')

In [ ]:
df_stats = df_stats.merge(df_results,how='right',on=['building','meter'])

In [ ]:
df_stats

In [ ]:
df= df_results.filter(regex='RMSE')
df.head()

## Time Series Split Cross Validation

In [ ]:
tss =  TimeSeriesSplit(n_splits=4,gap=4,test_size=14,max_train_size=120) #too generic
rmse = []
count = 1

results = pd.DataFrame()


# Write function that rolls next pd.delta?

for train_index, test_index in tss.split(df_train): 
    
    cv_train, cv_test = df_train.iloc[train_index]['meter_reading'], df_train.iloc[test_index]['meter_reading']
    model = pm.auto_arima(cv_train,trace=True,n_fits=20)
    print(model.summary())
    
    # 
    predicts = model.predict(n_periods=14)
    true_values = cv_test.values
    error = ((sqrt(mean_squared_error(predicts,true_values))))
    model.plot_diagnostics()
    rmse.append(error)
    print(error)
print('RMSE')
print(np.mean(rmse))

# Rolling Forecast ARIMA
### Walk-forward validation

In [ ]:
model.plot(forecast)

In [ ]:

#             # Fill Nas
#             #Counts of null values 
#             # Backfill with means
#             if np.max(X_train.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_train_Backfill'] = backfill
#                 if backfill == 'mean':
#                     X_train.fillna(X_train.mean(), inplace=True)

#             # Backfill with mean
#             if np.max(X_test.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     X_test.fillna(X_test.mean(), inplace=True)
                    
#             # Backfill with mean
#             if np.max(y_test.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     y_test.fillna(X_test.mean(), inplace=True)
                    
#             # Backfill with mean
#             if np.max(y_train.isnull().sum().sort_values(ascending=False)) > 0:
#                 stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
#                 if backfill == 'mean':
#                     y_train.fillna(X_train.mean(), inplace=True)